# Setup (Clone Repo)

Setup YOLO

In [1]:
!pip install ultralytics



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

Following cloning instructions from the orginal repo for APM

In [2]:
#try to clone the APM-David-Copy repo
%cd /content
!git clone https://github.com/davidg335/APM-David-Copy.git
%cd APM-David-Copy/
! ls

/content
Cloning into 'APM-David-Copy'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 242 (delta 11), reused 9 (delta 4), pack-reused 217 (from 1)
Receiving objects: 100% (242/242), 24.68 MiB | 15.54 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/APM-David-Copy
APM_person_ID.ipynb  interpolate.py	    README.md
assets		     lawnmower_images.zip   single_token_segmentation
david.jpeg	     misc_scripts	    visualize_coco.py
download.sh	     model.py
flop_analysis	     predict_test_image.py


In [3]:
#make sure local is up to date with main branch
!git pull origin main


From https://github.com/davidg335/APM-David-Copy
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
!wget "https://www.cs.ucf.edu/~rmodi/apm/model_15.pth"
# !wget https://www.cs.ucf.edu/~rmodi/apm/data.zip
!mkdir checkpoints
!mv model_15.pth checkpoints/
# !unzip data.zip


--2025-06-15 20:16:34--  https://www.cs.ucf.edu/~rmodi/apm/model_15.pth
Resolving www.cs.ucf.edu (www.cs.ucf.edu)... 132.170.216.243
Connecting to www.cs.ucf.edu (www.cs.ucf.edu)|132.170.216.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217396607 (207M)
Saving to: ‘model_15.pth’

model_15.pth        100%[===================>] 207.33M  67.6MB/s    in 3.1s    

2025-06-15 20:16:37 (67.6 MB/s) - ‘model_15.pth’ saved [217396607/217396607]



Unzip testing/training images

```
# This is formatted as code
```



In [5]:
!unzip lawnmower_images.zip -d lawnmower_images


Archive:  lawnmower_images.zip
   creating: lawnmower_images/lawnmower_images/
   creating: lawnmower_images/lawnmower_images/test/
  inflating: lawnmower_images/lawnmower_images/test/frames_0001.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0002.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0003.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0004.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0005.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0006.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0007.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0008.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0009.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0010.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0011.jpg  
  inflating: lawnmower_images/lawnmower_images/test/frames_0012.jpg  
  inflating: lawnmower_image

Model accuracy

In [6]:
#model = Model().cuda()
#x = torch.randn(4,3,448,448)
#x_avg = torch.randn(4,3,32,32)
#feat = torch.randn(4,1024,1024)
#loss, feat_loss, rgb_loss, feat_out, rgb_out = model.forward_wrapper(x,x_avg, feat)
#print(feat_loss, rgb_loss,feat_out.shape, rgb_out.shape)

# model.py



In [7]:
import os
import torch
import numpy as np
# import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import glob
from pathlib import Path
import cv2
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import torch.nn as nn
import math


def positionalencoding2d(d_model, height, width):
    """
    :param d_model: dimension of the model
    :param height: height of the positions
    :param width: width of the positions
    :return: d_model*height*width position matrix
    """
    if d_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(d_model))
    pe = torch.zeros(d_model, height, width)
    # Each dimension use half of d_model
    d_model = int(d_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) *
                         -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

    return pe.cuda()


class Model(nn.Module):
    # h,w of the image which will be fwd pass.
    # coordinate based query
    def __init__(self, hidden_dim = 1024, h = 32, w = 32, fwd_chunk_size = 16):

        super(Model, self).__init__()

        self.fc1 = nn.Linear(2*hidden_dim, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,4096)
        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048,1024)

        self.feat_proj_head = nn.Linear(1024, 1024)
        self.rgb_head_1 = nn.Linear(hidden_dim*3, 256) #input is x position vector, "DNA" vector T, and feature vector output for that pixel feat (the output of the MLP)
        self.rgb_head_2 = nn.Linear(256,256)
        self.rgb_head_3 = nn.Linear(256,3)

        #initialize positional encoding
        self.pos = positionalencoding2d(hidden_dim, h,w) #to break input coordinate symmetry
        self.h, self.w = h,w
        self.fwd_chunk_size = fwd_chunk_size

        #init a single patch size
        #will operate on 448 by 448 to get information into the columns
        self.patch_size = 14
        self.stride = 14
        self.conv1 = nn.Conv2d(3, 1, kernel_size=self.patch_size, stride=self.stride)

    def forward_chunk(self, x):
        x_pos = x#contains the whole cortical column stack

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.fc5(x)
        x = F.relu(x)

        feat = self.feat_proj_head(x)

        rgb = F.relu(self.rgb_head_1(torch.cat([feat,x_pos],1))) #breaks rgb output symmetry
        rgb = F.relu(self.rgb_head_2(rgb))
        rgb = F.relu(self.rgb_head_3(rgb))


        return feat, rgb # feat is the feature which is produced

    #x : b,c,h,w
    # feat: b (h w) d

    def forward_wrapper(self,x, x_avg, feat):
        x,x_avg,feat = x.cuda(), x_avg.cuda(),feat.cuda() #moves the variables from the CPU to the GPU

        b,c,h,w = x.shape #batch size (number of images), channel number (3 for RGB), height of image, width of image
        feat = rearrange(feat, 'b (h w) d -> b d h w', h = self.h, w = self.w)

        #trigger the copying of the average latent feature like dna
        summary_feat = self.conv1(x)
        summary_feat = rearrange(summary_feat, 'b c h w -> b (h w) c')#still need to resolve symmetry between locations in the cortical column
        summary_feat = summary_feat.squeeze(-1)
        summary_feat = repeat(summary_feat, 'b d -> b d h w', h = self.h, w = self.w) #squeezed the perceptual information into the column

        # print("summary feat", feat.shape, summary_feat.shape)

        pos = self.pos  #d h w
        pos = repeat(pos, 'd h w -> b d h w', b = b)

        input_feat = torch.cat([summary_feat, pos], dim=1) #along d dimension, break identity symmetry at same input location

        #batchify the entire forward pass
        input_feat = rearrange(input_feat, 'b d h w -> (b h w) d')
        target_feat = rearrange(feat, 'b d h w -> (b h w) d')
        target_rgb = rearrange(x_avg, 'b c h w -> (b h w) c')

        chunk_size = self.fwd_chunk_size
        n_chunks = input_feat.shape[0] // chunk_size
        if input_feat.shape[0] % chunk_size != 0:
            n_chunks += 1
        n_forwards = 0
        for i in range(n_chunks):

            start = i*chunk_size
            end = min((i+1)*chunk_size, input_feat.shape[0])
            input_feat_chunk = input_feat[start:end]

            feat_chunk, rgb_chunk = self.forward_chunk(input_feat_chunk)

            n_forwards+=1
            if i == 0:
                feat_out = feat_chunk
                rgb_out = rgb_chunk
            else:
                feat_out = torch.cat([feat_out, feat_chunk], dim=0)
                rgb_out = torch.cat([rgb_out, rgb_chunk], dim=0)

        feat_loss = F.mse_loss(feat_out, target_feat)

        rgb_loss = F.mse_loss(rgb_out, target_rgb)
        loss = feat_loss + rgb_loss

        rgb_out = rearrange(rgb_out, '(b h w) c -> b c h w', b = b, h = self.h, w = self.w)

        return loss, feat_loss, rgb_loss, feat_out, rgb_out



    # used for phasing one image into another. This is not needed for the NN architecture. It is a separate function.
    def interpolate_function(self,x1,x2, n_interpolations = 100):
        # no gradient flow required
        with torch.no_grad():
            x1,x2 = x1.cuda(), x2.cuda()
            b,c,h,w = x1.shape

            # f1,f2 are summary vectors
            f1,f2 = self.conv1(x1), self.conv1(x2)
            f1 = rearrange(f1,'b c h w -> b (h w) c')
            f2 = rearrange(f2,'b c h w -> b (h w) c')
            f1 = f1.squeeze(-1) #single col vector
            f2 = f2.squeeze(-1) #single col vector

            pos = self.pos  #d h w
            pos = repeat(pos, 'd h w -> b d h w', b = b)

            #generate n_interpolations between f1 and f2
            interpolation_vectors = []
            for i in range(n_interpolations):
                interpolation_vectors.append(f1 + (f2 - f1) * i / n_interpolations)

            output_feats = []
            output_rgbs = []

            for i in range(n_interpolations):
                print("interpolation", i, "/", n_interpolations)
                summary_vector = interpolation_vectors[i]
                print("interpolation shape", summary_vector.shape)
                summary_vector = repeat(summary_vector, 'b d -> b d h w', h = self.h, w = self.w) #repeat column vector to all locations
                summary_vector = summary_vector.cuda()
                input_feat = torch.cat([summary_vector, pos], dim=1) #along d dimension
                #batchify the entire forward pass
                input_feat = rearrange(input_feat, 'b d h w -> (b h w) d')

                chunk_size = self.fwd_chunk_size
                n_chunks = input_feat.shape[0] // chunk_size
                if input_feat.shape[0] % chunk_size != 0:
                    n_chunks += 1
                for j in range(n_chunks):
                    # print("chunk", i, "/", n_chunks)
                    start = j*chunk_size
                    end = min((j+1)*chunk_size, input_feat.shape[0])
                    input_feat_chunk = input_feat[start:end]
                    feat_chunk, rgb_chunk = self.forward_chunk(input_feat_chunk)
                    #rgb is not needed right now
                    #rgb_out = None
                    if j == 0:
                        feat_out = feat_chunk
                        rgb_out = rgb_chunk
                    else:
                        feat_out = torch.cat([feat_out, feat_chunk], dim=0)
                        rgb_out = torch.cat([rgb_out, rgb_chunk], dim=0)

                feat_out  = feat_out.cpu().detach().numpy()
                output_feats.append(feat_out)
                output_rgbs.append(rgb_out)
                feat_out = None #next vector interpolation happens here
                rgb_out = None

        print("getting ready to return....")
        # exit(1)
        return output_feats,output_rgbs

    def predict_image(self, x):
        x = x.cuda()
        b,c,h,w = x.shape

        summary_feat = self.conv1(x)
        summary_feat = rearrange(summary_feat, 'b c h w -> b (h w) c')
        summary_feat = summary_feat.squeeze(-1)
        summary_feat = repeat(summary_feat, 'b d -> b d h w', h = self.h, w = self.w)

        pos = self.pos
        pos = repeat(pos, 'd h w -> b d h w', b = b)

        input_feat = torch.cat([summary_feat, pos], dim=1)
        input_feat = rearrange(input_feat, 'b d h w -> (b h w) d')

        chunk_size = self.fwd_chunk_size
        n_chunks = input_feat.shape[0] // chunk_size
        if input_feat.shape[0] % chunk_size != 0:
            n_chunks += 1
        n_forwards =0
        for i in range(n_chunks):
            start = i*chunk_size
            print("rmodi", chunk_size, input_feat.shape[0], type(chunk_size), type(input_feat.shape[0]))

            # end = min(int((i+1)*chunk_size), 1024)
            end = min((i+1)*chunk_size, input_feat.shape[0])
            input_feat_chunk = input_feat[start:end]
            feat_chunk, rgb_chunk = self.forward_chunk(input_feat_chunk)

            if i == 0:
                feat_out = feat_chunk
                rgb_out = rgb_chunk
            else:
                feat_out = torch.cat([feat_out, feat_chunk], dim=0)
                rgb_out = torch.cat([rgb_out, rgb_chunk], dim=0)
            n_forwards+=1
        # print("no of forwards", n_forwards)
        # print("network just made 32*32 forward passes for 1024 pixels.  no need to keep all pixels of input percept in memory. A PATCH IS A BATCH. HAIL HINTON!!! HAIL GLOM!!!! :-)")
        # exit(1)
        feat_out  = feat_out.cpu().detach().numpy()
        rgb_out = rgb_out.cpu().detach().numpy()
        return feat_out,rgb_out



# predict_test_image.py file

Runs the model on a test image

In [8]:
#predict_test_image.py file

# author: rmodi
# performs prediction on any test image

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import cv2
from einops import rearrange, reduce, repeat
# from dataloader import PerceptionFieldDataset
# from model import Model
import copy

from sklearn.manifold import TSNE


# if __name__ == '__main__':

h,w = 32,32 # dimension of the input image into the folding block
d = 1024 #size of the feature vector that will be attached to each pixel
patch_size = 14
fwd_chunk_size = 1
batch_size = 1
num_workers = 8
num_epochs = 100000
lr = 0.0001
device = 'cuda'

save_path = Path('./checkpoints')
save_path.mkdir(parents=True, exist_ok=True)
model_save_path = Path('checkpoints/model_15.pth')


img_path = 'david.jpeg'
img_name=img_path.split('.')[0]
img = cv2.imread(img_path)
print("read")

img = cv2.resize(img, (448,448))
orig_img = copy.deepcopy(img)
min_img = np.min(img)
max = np.max(img)
img = (img - min_img) / (max - min_img) #do min max scaling
x = torch.from_numpy(img).permute(2,0,1).float()
x = x.unsqueeze(0)
x = x.to(device)
print("x shape", x.shape)
########################################################################################
#build model
model = Model(hidden_dim = d, h = h, w = w, fwd_chunk_size = fwd_chunk_size).to(device)

#load model
model.load_state_dict(torch.load(model_save_path), strict = True)
print("loaded model")

#perform prediction
model.eval()

#####################################################################################
# exit(1)
#recreating a test image using the model created
with torch.no_grad(): # indicates that don't want weights to be updated via GD when testing
    feat_out,rgb_out = model.predict_image(x) #rgb_out is reconstructed image 32x32x3, not sure what feat_out is, should be 1000x1000
    print("feat out", feat_out.shape)
x = TSNE(n_components=3, #output has three dimensions
         learning_rate='auto', #algorithm chooses learning rate
              init='random', #embedding intialization is random
          perplexity=3).fit_transform(feat_out) #perplexity is num nearest neighbors considered
#x is the feature vector reduced to three dimensions (rgb)


x = rearrange(x, '(h w) c -> h w c', h = 32, w = 32)

#make x be normalized into rgb scale 0 to 255
x = (x - np.min(x)) / (np.max(x) - np.min(x))
x = x * 255
x = x.astype(np.uint8)

#making the output image black and white
rgb_out = (rgb_out - np.min(rgb_out)) / (np.max(rgb_out) - np.min(rgb_out))
rgb_out = rgb_out * 255
rgb_out = rearrange(rgb_out, '(h w) c -> h w c', h = 32, w = 32)

orig_img = cv2.resize(orig_img, (32,32)) #downsize image to fit input size 32x32 for model
to_save =  np.concatenate((orig_img, rgb_out, x), axis = 1) #original image| predicted rgb| predicted feat.
print("saving....")
cv2.imwrite('./viz_'+img_name+'.jpg', to_save) #save "david.jpg" input as three image output "viz_david.jpg" with original image, grayscale reconstruction, segmentation



read
x shape torch.Size([1, 3, 448, 448])
loaded model
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rmodi 1 1024 <class 'int'> <class 'int'>
rm

True

# single token segmentation

## model.py
Definition of Model class (the APM model)

In [9]:
# i am forever humbled and grateful to geoff hinton for sharing his glom and forward forward paper with all of us.
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
from pathlib import Path
import cv2
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import torch.nn as nn
import math


def positionalencoding2d(d_model, height, width):
    """
    :param d_model: dimension of the model
    :param height: height of the positions
    :param width: width of the positions
    :return: d_model*height*width position matrix
    """
    if d_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(d_model))
    pe = torch.zeros(d_model, height, width)
    # Each dimension use half of d_model
    d_model = int(d_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) *
                         -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

    return pe.cuda()


class Model(nn.Module):
    # h,w of the image which will be fwd pass.
    # coordinate based query
    def __init__(self, hidden_dim = 1024, h = 32, w = 32, fwd_chunk_size = 16):

        super(Model, self).__init__()

        self.fc1 = nn.Linear(2*hidden_dim, 4096)

        self.fc4 = nn.Linear(4096, 2048)
        self.fc5 = nn.Linear(2048,1024)

        self.feat_proj_head = nn.Linear(1024, 768)
        self.rgb_head_1 = nn.Linear(2816, 256)
        self.rgb_head_2 = nn.Linear(256,256)
        self.rgb_head_3 = nn.Linear(256,3)

        #initialize positional encoding
        self.pos = positionalencoding2d(hidden_dim, h,w) #to break input coordinate symmetry
        self.h, self.w = h,w
        self.fwd_chunk_size = fwd_chunk_size

        #init a single patch size
        #will operate on 448 by 448 to get information into the columns
        self.patch_size = 14
        self.stride = 14
        self.conv1 = nn.Conv2d(3, 1, kernel_size=self.patch_size, stride=self.stride)
        self.norm = nn.LayerNorm(hidden_dim)

        self.little_self_attention = nn.MultiheadAttention(768, 8, batch_first=True)
        self.little_query = nn.Parameter(torch.randn(768))
        self.pos_output = positionalencoding2d(768, h,w) #loving attention at the output of the cortical stack.


    def forward_chunk(self, x):

        # print("slimmed down net..")
        x_pos = x#contains the whole cortical column stack

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc4(x)
        x = F.relu(x)
        x = self.fc5(x)
        x = F.relu(x)

        feat = self.feat_proj_head(x)

        rgb = F.relu(self.rgb_head_1(torch.cat([feat,x_pos],1))) #breaks rgb output symmetry
        rgb = F.relu(self.rgb_head_2(rgb))
        rgb = F.relu(self.rgb_head_3(rgb))



        return feat, rgb # feat is the feature which is produced

    #x : b,c,h,w
    # feat: b (h w) d

    def forward_wrapper(self,x, x_avg, feat):
        x,x_avg,feat = x.cuda(), x_avg.cuda(),feat.cuda()

        b,c,h,w = x.shape

        #trigger the copying of the average latent feature like dna
        summary_feat = self.conv1(x)
        summary_feat = rearrange(summary_feat, 'b c h w -> b (h w) c')#still need to resolve symmetry between locations in the cortical column
        summary_feat = summary_feat.squeeze(-1)
        summary_feat = repeat(summary_feat, 'b d -> b d h w', h = self.h, w = self.w) #squeezed the perceptual information into the column



        pos = self.pos  #d h w
        pos = repeat(pos, 'd h w -> b d h w', b = b)

        input_feat = torch.cat([summary_feat, pos], dim=1) #along d dimension, break identity symmetry at same input location

        #batchify the entire forward pass
        input_feat = rearrange(input_feat, 'b d h w -> (b h w) d')

        target_feat= feat
        target_rgb = rearrange(x_avg, 'b c h w -> (b h w) c')

        chunk_size = self.fwd_chunk_size
        n_chunks = input_feat.shape[0] // chunk_size
        if input_feat.shape[0] % chunk_size != 0:
            n_chunks += 1
        n_forwards = 0
        for i in range(n_chunks):

            start = i*chunk_size
            end = min((i+1)*chunk_size, input_feat.shape[0])
            input_feat_chunk = input_feat[start:end]

            feat_chunk, rgb_chunk = self.forward_chunk(input_feat_chunk)

            n_forwards+=1
            if i == 0:
                feat_out = feat_chunk
                rgb_out = rgb_chunk
            else:
                feat_out = torch.cat([feat_out, feat_chunk], dim=0)
                rgb_out = torch.cat([rgb_out, rgb_chunk], dim=0)


        feat_out = rearrange(feat_out, '(b h w) d -> b h w d', b = b, h = self.h, w = self.w)
        orig_feat_out = feat_out


        pos_out = repeat(self.pos_output, 'd h w -> b d h w', b = b)
        pos_out = rearrange(pos_out, 'b d h w -> b h w d')
        feat_out = feat_out + pos_out
        feat_out = rearrange(feat_out ,'b h w d -> b (h w) d')#(N,L,E q ​ ) when batch_first=True

        query = repeat(self.little_query, 'd -> b  d', b = b, )
        query = query.unsqueeze(1) #need one query only


        feat_out, _ = self.little_self_attention(query, feat_out, feat_out)

        feat_out = feat_out.squeeze(1)

        feat_loss = F.mse_loss(feat_out, target_feat)
        rgb_loss = 0

        loss = feat_loss + rgb_loss

        rgb_out = rearrange(rgb_out, '(b h w) c -> b c h w', b = b, h = self.h, w = self.w)

        return loss, feat_loss, rgb_loss, feat_out, rgb_out,orig_feat_out



## model_tta.py

This code runs the train and test code through YOLO and outputs the labels and feature vectors for the train and test code.

In [10]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO

# Set up transform (resize and convert to tensor)
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

# Custom dataset to load images from a folder
class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        self.image_files.sort()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.folder_path, self.image_files[idx])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.image_files[idx]  # return filename too for reference

# Wrapper model that uses YOLO for inference
#base class is torch.nn.Module
class ModelTTA(torch.nn.Module):
    def __init__(self, model_path='yolov5s.pt', device='cuda'):
        super(ModelTTA, self).__init__()
        print("Loading YOLO model...")
        self.device = device
        self.model = YOLO(model_path)
        print("YOLO loaded.")

    def forward(self, img):
        # Undo ToTensor transform: convert tensor to numpy array for YOLO
        img_np = img.mul(255).byte().permute(1, 2, 0).cpu().numpy()
        results = self.model(img_np)
        return results

# Run inference on a dataset
def run_inference_on_folder(folder_path, model):
    dataset = ImageFolderDataset(folder_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

    for idx, (img, filename) in enumerate(dataloader):
        print(f"Processing {filename[0]} ({idx+1}/{len(dataset)})")
        img = img[0].to('cuda')  # remove batch dimension

        results = model(img)

        for res in results:
            boxes = res.boxes
            if boxes is not None and len(boxes) > 0:
                for box in boxes:
                    cls_id = int(box.cls[0])
                    conf = float(box.conf[0])
                    print(f"Detected class {model.model.names[cls_id]} with confidence {conf:.3f}")
            else:
                print("No detections")
        print("-----")

# this script here will only run when this script is run directly and not when it is imported as a module into another scipt
# if this script is run directly, __name__ is automatically set to '__main__'
if __name__ == '__main__':
    model = ModelTTA().to('cuda')

    # Set your paths here
    train_path = '/content/APM-David-Copy/lawnmower_images/lawnmower_images/train'
    test_path = '/content/APM-David-Copy/lawnmower_images/lawnmower_images/test'

    print("Running on TRAIN set:")
    run_inference_on_folder(train_path, model)

    print("Running on TEST set:")
    run_inference_on_folder(test_path, model)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading YOLO model...
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 135MB/s] 


YOLO loaded.
Running on TRAIN set:
Processing frames_0151.jpg (1/48)

0: 640x640 1 person, 15.0ms
Speed: 12.4ms preprocess, 15.0ms inference, 334.3ms postprocess per image at shape (1, 3, 640, 640)
Detected class person with confidence 0.788
-----
Processing frames_0152.jpg (2/48)

0: 640x640 1 person, 15.0ms
Speed: 5.5ms preprocess, 15.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Detected class person with confidence 0.840
-----
Processing frames_0153.jpg (3/48)

0: 640x640 1 person, 14.9ms
Speed: 4.9ms preprocess, 14.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Detected class person with confidence 0.762
-----
Processing frames_0154.jpg (4/48)

0: 640x640 1 person, 15.0ms
Speed: 5.3ms preprocess, 15.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Detected class person with confidence 0.733
-----
Processing frames_0155.jpg (5/48)

0: 640x640 1 person, 14.9ms
Speed: 5.4ms preprocess, 14.9ms inference, 1.4ms postproces

In [11]:
# i am forever humbled and grateful to geoff hinton for sharing his glom and forward forward paper with all of us.
import os
import torch
import numpy as np
# import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import glob
from pathlib import Path
import cv2
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import torch.nn as nn
import math

from model import Model
import clip
from torchvision import transforms


transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]) # these are standard mean and standard deviation values for RGB values from large image datasets


class ModelTTA(nn.Module):
    def __init__(self, hidden_dim = 1024, h = 32, w = 32, fwd_chunk_size = 16, num_classes= 10, backbone_path = None):

        super(ModelTTA, self).__init__()

        device = 'cuda'
        #load the clip model
        self.clip_teacher, self.preprocess = clip.load('ViT-L/14', device)


        print("init model")
        self.backbone = Model(hidden_dim = hidden_dim, h = h, w = w, fwd_chunk_size = fwd_chunk_size) #backbone is an instance of "Model"
        print("loaded backbone")
        print("loaded classification")
        self.h = h
        self.w = w
        self.num_classes = num_classes



    def forward(self, img,cls_id, image_features):#, x, x_avg, feat): #image_features and cls_id are the feature vector
        x = transform(img)
        # print("x is", x.shape)
        x = repeat(x, 'c h w -> b c h w', b = 8) #prepare for forward passes.
        x_avg = F.avg_pool2d(x, kernel_size=(14,14), stride=(14,14))


        #forward pass through backbone, and force it to align
        loss, feat_loss, rgb_loss, feat_out, rgb_out,orig_feat_out = self.backbone.forward_wrapper(x, x_avg, image_features) #image features of clip are target for our model

        return loss, feat_out,orig_feat_out


if __name__ == '__main__':

    # print("testing dataset", clip.available_models())
    # exit(1)
    from torchvision.datasets import CIFAR100
    import os
    dataset = CIFAR100(root=os.path.expanduser("~/data"), download=True, train=False)
    model = ModelTTA().cuda()
    for idx in range(len(dataset)):

        print("doing", idx+1, "of", len(dataset))
        image, class_id = dataset[idx]
        loss, status = model(image,class_id, dataset,)
        # print("image is", image.shape)
        print("loss is", loss,status)
        # exit(1)
    # x = torch.randn(4,3,448,448)
    # x_avg = torch.randn(4,3,32,32)
    # feat = torch.randn(4,1024)
    # model(x,x_avg, feat)





ModuleNotFoundError: No module named 'clip'

## train_tta.py

I modified this code to use YOLO instead of CLIP.

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
from pathlib import Path
import cv2
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import torch.nn as nn
import math
from torchvision import transforms
import torchvision
from torch.cuda.amp import autocast, GradScaler
from sklearn.manifold import TSNE

# NEW: Import YOLOv5
from ultralytics import YOLO
model = YOLO("yolo11n.pt")

scaler = GradScaler()

# Hyperparameters
h, w = 32, 32
d = 1024
n_iters_tta = 1200
n_samples_fwd = 4
patch_size = 14
fwd_chunk_size = 512
batch_size = 32 * 2
num_workers = 8
num_epochs = 15
lr = 1e-4
device = 'cuda'
train_print_freq = 50
is_train = False
target_trues = 1200

# Load YOLOv5 model (download yolov5s.pt if needed)
model = YOLO("yolo11n.pt")

def randomize_weights_student(model):
    for param in model.parameters():
        print("randomizing!!!")
        nn.init.normal_(param.data, mean=0, std=0.01)

model_tta = ModelTTA(hidden_dim=1024, h=32, w=32, fwd_chunk_size=16, num_classes=2, backbone_path=None).cuda()

# Use CIFAR10 for now, modify later if needed
dataset = torchvision.datasets.CIFAR10(root='~/data', train=is_train, download=True, transform=None)

import torch.optim.lr_scheduler as lr_scheduler

optimizer = torch.optim.Adam(model_tta.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

correct, n_preds = 0, 0

def dump_island(orig_feat_out, i, iter):
    orig_feat_out = orig_feat_out.detach().cpu().numpy()
    orig_feat_out = orig_feat_out[0]
    feat_out = rearrange(orig_feat_out, 'h w d -> (h w) d')
    feat = TSNE(n_components=1, learning_rate='auto',
                init='random', perplexity=3).fit_transform(feat_out)
    feat = (feat - np.min(feat)) / (np.max(feat) - np.min(feat))
    feat = feat.squeeze(-1)
    feat = rearrange(feat, '(h w)  -> h w ', h=32, w=32)

    feat = repeat(feat, 'h w -> h w c', c=3)
    feat = feat * 255
    feat = feat.astype(np.uint8)
    feat = np.concatenate([orig_img, feat], axis=1)
    save_str = '{}_{}.png'.format(i, iter)
    cv2.imwrite(save_str, feat)
    print("write done")

idx = 200
for i in range(idx, len(dataset)):
    print("discarded backbone and reloaded weights from scratch")
    randomize_weights_student(model_tta.backbone)
    print("randomized")
    n_trues = 0
    is_last_True = False

    img, label = dataset[i]
    orig_img = np.array(img)

    # Convert to BGR for OpenCV compatibility
    img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    # Run YOLO inference
    results = yolo.predict(img_bgr)

    # Check if any detection is 'person' (class 0 in COCO)
    contains_person = any(int(cls) == 0 for cls in results.pred[0][:, 5])

    # Create binary label based on YOLO detection
    yolo_label = 1 if contains_person else 0

    optimizer.zero_grad()
    was_correctly_predicted = False
    break_count = 0

    for iter in range(n_iters_tta):
        print("iter", iter, '/', n_iters_tta)

        with autocast():
            # Forward TTA model
            loss, feat_out, orig_feat_out = model_tta(img, yolo_label)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            print("loss scaled", loss)

        if iter % 50 == 0:
            print("going for routing")
            dump_island(orig_feat_out, i, iter)

    break  # REMOVE this break to run full loop


Chat output for classifying person vs no person

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

class TokenRegressor(nn.Module):
    def __init__(self, input_dim=3*224*224, token_dim=256):  # Match YOLO token size
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, token_dim)
        )

    def forward(self, x):
        return self.net(x)

# Loss: L2 between predicted and true YOLO person token
mse_loss = nn.MSELoss()

def train_model(model, train_loader, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, true_tokens in tqdm(train_loader):
            images, true_tokens = images.to(device), true_tokens.to(device)
            pred_tokens = model(images)

            loss = mse_loss(pred_tokens, true_tokens)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Avg Train Loss = {total_loss / len(train_loader):.4f}")


Here is the original train_tta file:

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
from pathlib import Path
import cv2
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import torch.nn as nn
import math
from torchvision import transforms
import torchvision
from model_tta import ModelTTA
import clip
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()
from sklearn.manifold import TSNE


h,w = 32,32
d = 1024
n_iters_tta = 1200 #number of iterations for test time training for each sample
n_samples_fwd = 4
patch_size = 14
fwd_chunk_size = 512
batch_size = 32*2
num_workers = 8
num_epochs = 15
lr = 1e-4
device = 'cuda'
train_print_freq = 50
is_train = False
target_trues = 1200

clip_teacher, preprocess = clip.load('ViT-L/14', device)

def randomize_weights_student(model):
    for param in model.parameters():
        print("randomizing!!!")
        nn.init.normal_(param.data, mean=0, std=0.01)  # Initialize with random values from a normal distribution


model_tta = ModelTTA(hidden_dim = 1024, h = 32, w = 32, fwd_chunk_size = 16, num_classes= 10, backbone_path = None).cuda()
dataset = torchvision.datasets.CIFAR10(root = '~/data', train=is_train, download=True, transform=None)

import torch.optim.lr_scheduler as lr_scheduler

optimizer = torch.optim.Adam(model_tta.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

correct,n_preds =0,0



def dump_island(orig_feat_out, i, iter):
    orig_feat_out = orig_feat_out.detach().cpu().numpy()
    orig_feat_out = orig_feat_out[0]
    feat_out = rearrange(orig_feat_out, 'h w d -> (h w) d')
    feat = TSNE(n_components=1, learning_rate='auto',
                 init='random', perplexity=3).fit_transform(feat_out)
    feat = (feat - np.min(feat)) / (np.max(feat) - np.min(feat))
    feat = feat.squeeze(-1)
    feat = rearrange(feat, '(h w)  -> h w ', h = 32, w = 32)

    feat = repeat(feat, 'h w -> h w c', c = 3)
    feat = feat * 255
    feat = feat.astype(np.uint8)
    feat = np.concatenate([orig_img, feat], axis = 1)
    save_str = '{}_{}.png'.format(i, iter)
    cv2.imwrite(save_str, feat)
    print("write done")
idx = 200
for i in range(idx, len(dataset)):
    # adapt the model everytime
    print("discarded backbone and reloaded weights from scratch")
    randomize_weights_student(model_tta.backbone)
    print("randomized")
    n_trues=0
    is_last_True = False
    # exit(1)
    img, label = dataset[i]
    orig_img = np.array(img)

    # print("image shape", orig_img.shape)
    # exit(1)
    optimizer.zero_grad()
    was_correctly_predicted = False
    break_count =0
    for iter in range(n_iters_tta):

        print("iter", iter, '/', n_iters_tta)
        image_input = preprocess(img).unsqueeze(0).to('cuda')
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in dataset.classes]).to('cuda')
        with autocast():
            with torch.no_grad():
                image_features = clip_teacher.encode_image(image_input)
                text_features = clip_teacher.encode_text(text_inputs)


            image_features = repeat(image_features, 'b d-> b b1 d', b1 = 8)
            image_features = rearrange(image_features, 'b b1 d -> (b b1) d')

        # with autocast():
            loss, feat_out,orig_feat_out = model_tta(img, label, image_features) #forward pass with the one image img, the label/image features output from teacher model,
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            # loss.backward()
            # optimizer.step()
            print("loss scaled", loss)

            image_features = feat_out[0]
            # text_features = text_features.to(torch.float32)
            # image_features = image_features.to(torch.float32)
            image_features = image_features.unsqueeze(0)

            # print("image features", image_features.shape, text_features.shape, image_features.dtype, text_features.dtype)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            values, indices = similarity[0].topk(5)
            index = indices[0]


        if iter%50==0:
            print("going for routing")
            dump_island(orig_feat_out, i, iter)






# Visualizations

## Visualize output of model_tta.py

In [ ]:
import os
import random
import torch
import numpy as np
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

# Path to your image folder
folder_path = '/content/APM-David-Copy/lawnmower_images/lawnmower_images/train'

# Load YOLO model
model = YOLO('yolov5s.pt')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define transformation (optional resize)
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])

# Select 10 random images
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(image_files)
selected_images = image_files[:9]

# Set up the grid
fig, axes = plt.subplots(3, 3, figsize=(20, 8))

for i, img_name in enumerate(selected_images):
    img_path = os.path.join(folder_path, img_name)
    img_pil = Image.open(img_path).convert("RGB")
    img_np = np.array(img_pil)

    # Run inference
    results = model(img_np)

    # Determine if 'person' is detected
    found_person = False
    for res in results:
        boxes = res.boxes
        if boxes is not None and len(boxes) > 0:
            for box in boxes:
                cls_id = int(box.cls[0])
                class_name = model.model.names[cls_id]
                if class_name.lower() == 'person':
                    found_person = True
                    break
        if found_person:
            break

    label = "Person" if found_person else "No Person"

    # Plot
    ax = axes[i //3, i % 3]
    ax.imshow(img_pil)
    ax.set_title(label, fontsize=14, color='green' if found_person else 'red')
    ax.axis('off')

plt.tight_layout()
plt.show()
